In [ ]:
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Here to pre-process the data input
training_places = []
training_labels = []
testing_places = []
testing_labels = []

with open("/tmp/amusement_park.json", 'r') as f:
    datastore = json.load(f)

# print(datastore['results'][2]['name'])
for item in datastore['results']:
  training_places.append(item['name'])

training_places

['Kampung Wisata Sosromenduran',
 'Kids Fun Toko Progo',
 'Taman Air Menari',
 'Kampung MARKISA Blunyahrejo',
 'Cemerlang Abadi. CV']

In [ ]:
vocab_size = 12000
embedding_dim = 16
max_length = 24
trunc_type = 'post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_places)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_places)
training_padded = pad_sequences(sequences, maxlen=max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(training_places)
testing_padded = pad_sequences(testing_sequences)

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size,
                            embedding_dim,
                            input_length=max_length),
  tf.keras.layers.Conv1D(64, 5, activation='relu'),
  tf.keras.layers.GlobalMaxPooling1D(),
  tf.keras.layers.Dense(48, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax') 
])

# kategori: **Budaya, Sejarah, Cagar Alam, Maritim, Religi**

model.compile(loss="categorical_crossentropy",
              metrics=["accuracy"],
              optimizer='adam')

model.summary()

In [ ]:
num_epochs = 10

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

history = model.fit(training_padded, training_labels,
                    epochs=num_epochs,
                    validation_data=(testing_padded,testing_labels),
                    verbose = 1)